# Tree to png

In [2]:
from treelib import Tree, Node
import pydot as pd

In [2]:
t = Tree()
t.create_node('root','root')
t.create_node('ch1','ch1',parent='root')
t.create_node('ch2','ch2',parent='root')

Node(tag=ch2, identifier=ch2, data=None)

In [3]:
t.to_graphviz('test.gv')
dot = pd.graph_from_dot_file('test.gv')[0]

In [4]:
dot.write_png('test.png')

# Optimizing score and heuristic

starting with evaluating the performance of regex

In [4]:
import re
from time import perf_counter_ns
from model.grid import Grid

In [4]:
reg = re.compile("(1(?!1{3,})|0(?!0{3,})){4,}")

avg = 0
max = 0
for i in range(4000000):
    start = perf_counter_ns()
    j = 0
    matches = reg.finditer("00001201010")
    for match in matches:
        j += 1
    d = (perf_counter_ns() - start) / 1e6
    avg = (avg +  d)/ 2
    if max < d: max = d
print("Average =",avg,"ms")
print("Max =",max,"ms")

Average = 0.005135909966863656 ms
Max = 35.9908 ms


The performance of regex is in the range of microseconds. It is not the problem, these unrealistic numbers are coming from somewhere else in the score and heuristic functions

## using regex for scores

In [16]:
g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")

avg = 0.23693513147081263 ms.
max = 3.5371 ms


In [17]:
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [1,2,3,4,5,6,7,8,9,10,11,12,13]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

1.659 seconds.
11.61 seconds.
81.269 seconds.
568.881 seconds.
3982.169 seconds.
27875.181 seconds.
195126.269 seconds.
1365883.883 seconds.
9561187.18 seconds.
66928310.259 seconds.
468498171.813 seconds.
3279487202.694 seconds.
22956410418.859 seconds.


Minor optimization: which representation of the elements is more efficient at conversion to strings.

In [18]:
avgchr = 0
avgint = 0
for i in range(10000000):
    t = perf_counter_ns()
    chr(65)
    avgchr = ((avgchr + (perf_counter_ns() - t) / 1e6)) / 2
    t = perf_counter_ns()
    str(1)
    avgint = (avgint + ((perf_counter_ns() - t) / 1e6)) /2
print(avgchr)
print(avgint)

0.00016249998956445748
0.0003000015735742899


using bytes is better than ints

In [1]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")

avg = 0.20091451739831348 ms.
max = 1.103 ms


In [3]:
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

1.406 seconds.
9.845 seconds.
68.914 seconds.
482.396 seconds.
3376.77 seconds.
23637.392 seconds.
165461.744 seconds.
1158232.211 seconds.
8107625.476 seconds.
56753378.33 seconds.
397273648.309 seconds.
2780915538.16 seconds.
19466408767.123 seconds.


## replacing regex with manual string search

In [1]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")

avg = 0.14025233685007482 ms.
max = 2.1745 ms


In [2]:
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

0.982 seconds.
6.872 seconds.
48.107 seconds.
336.746 seconds.
2357.221 seconds.
16500.547 seconds.
115503.83 seconds.
808526.812 seconds.
5659687.682 seconds.
39617813.775 seconds.
277324696.422 seconds.
1941272874.953 seconds.
13588910124.673 seconds.
